In [1]:
import pickle
#from loadData import dataLoader
import tensorflow as tf
#from CalculateEER import SKGetEER as GetEER
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_curve, auc
from scipy import interp


In [2]:
def onehot(x,numlabels):
    t = [0 for i in range(numlabels)]
    t[x-1] = 1
    return t

In [190]:
def GetEER(y_score, y_test):
    #y_score = y_score.get_label()
    y_test = np.array(y_test.get_label()).reshape(len(y_score), 100)
    n_classes = y_score.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    missRate = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    missRate["micro"] = 1 - tpr["micro"]
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    missRate["macro"] = 1 - tpr["macro"]
    #print(np.dtype(min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])))
    return 'EER', min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])


In [191]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
data= pickle.load(open('/Users/chaitanya/Documents/python/keystrokes/data_augmentation/greyc1/PCA_GREYC_2X.pickle','rb'))


In [192]:
X = data['data']
X = np.array(X).reshape(len(X), 28)
y = np.array(data['labels']).reshape(len(X), 100)
#y_train = [onehot(x,100) for x in train_data['labels']]
#X_test = np.array(test_data['features']).reshape(len(test_data['features']), 1, 28, 1)
#y_test = [onehot(x,100) for x in test_data['labels']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=122)
print([len(x) for x in [X_test, y_test, X_train, y_train]])

y_train=np.reshape(y_train,(len(y_train)*100))
y_test = np.reshape(y_test, (len(y_test)*100))

[4445, 4445, 10371, 10371]


In [228]:
xg_train = xgb.DMatrix(X_train, label =  y_train)
xg_test = xgb.DMatrix(X_test, label =  y_test)
param = {}
param['objective'] = 'reg:logistic'
param['eta'] = 0.2
param['max_depth'] = 4
param['silent'] = 1
param['nthread'] = 10
param['num_class'] = 100
#param['nrounds'] = 100
param['eval_metric'] = 'rmse'
#param['']
n_classes = 100
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 100
bst = xgb.train(param, xg_train, num_round,  watchlist, feval = GetEER, maximize =False)



[0]	train-rmse:0.403134	test-rmse:0.403438	train-EER:0.132957	test-EER:0.162139
[1]	train-rmse:0.327693	test-rmse:0.328392	train-EER:0.104838	test-EER:0.133524
[2]	train-rmse:0.267941	test-rmse:0.26909	train-EER:0.09065	test-EER:0.120308
[3]	train-rmse:0.220238	test-rmse:0.221917	train-EER:0.081688	test-EER:0.112436
[4]	train-rmse:0.182004	test-rmse:0.184318	train-EER:0.070069	test-EER:0.104598
[5]	train-rmse:0.151337	test-rmse:0.154415	train-EER:0.063536	test-EER:0.102101
[6]	train-rmse:0.126728	test-rmse:0.130704	train-EER:0.056052	test-EER:0.098501
[7]	train-rmse:0.107085	test-rmse:0.112046	train-EER:0.048569	test-EER:0.09589


KeyboardInterrupt: 

In [226]:
param['objective'] = 'reg:logistic'
param['eta'] = 0.06
param['max_depth'] = 4
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 100
#param['nrounds'] = 100
param['eval_metric'] = 'error'
n_classes = 100
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 100
bst = xgb.train(param, xg_train, num_round,  watchlist, feval = GetEER, maximize =False)



[0]	train-error:0.004388	test-error:0.005744	train-EER:0.132957	test-EER:0.162139
[1]	train-error:0.004083	test-error:0.005447	train-EER:0.125219	test-EER:0.154485
[2]	train-error:0.004066	test-error:0.005372	train-EER:0.107163	test-EER:0.139094


KeyboardInterrupt: 

In [218]:
ytest = np.reshape(y_test,(len(X_test), 100))
pred = bst.predict(xg_test)
error_rate = np.sum(np.argmax(pred) != np.argmax(y_test))/np.shape(y_test)[0]

In [220]:
1-error_rate

0.9999977502812148

In [171]:
pred = np.reshape(pred,(len(pred),100))
pred = np.zeros((len(pred),100))

In [165]:
np.argmax(pred[1]), np.argmax(ytest[1])

(62, 62)

In [198]:
bst.save_model('0001.model')

In [224]:
bst = xgb.Booster({'nthread': 4})  # init model
bst.load_model('0001.model')  # load datan

In [205]:
pred1 = bst.predict(xg_test)

In [214]:
pred1== pred

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])